In [1]:
import requests
import pandas as pd
import json
import mysql.connector
from datetime import datetime, timedelta
import time
import nvda_config1

import twelvedata
from twelvedata import TDClient

In [2]:
# Including this login info just in case 
Cpassword = nvda_config1.mysql_password
td = TDClient(apikey=nvda_config1.twelve_api_nvda) 
base_url = 'https://api.twelvedata.com'

# Establish MySQL object
mydb = mysql.connector.connect(
    host="localhost",   
    user="root",         
    password=Cpassword,  
    database="nvdav1"   
)

mycursor = mydb.cursor()



In [27]:
# Read CSV files and create pandas dataframes
# Stock prices foor NVDA historical for 1 yeaar at 1 minute intervals
df_stock = pd.read_csv('C:/Users/micha/OneDrive/Desktop/NVDALinearLogisticAnalysis/stock_ticks.csv')
# technical inidcators foor NVDA historical for 1 yeaar at 1 minute intervals
df_ti = pd.read_csv('C:/Users/micha/OneDrive/Desktop/NVDALinearLogisticAnalysis/technical_indicators.csv')
# company info
df_nvda_info = pd.read_csv('C:/Users/micha/OneDrive/Desktop/NVDALinearLogisticAnalysis/nvda_profile.csv')
# new data for recent days downlaoded by the minute. Will have errors
df_recent_new = pd.read_csv('C:/Users/micha/OneDrive/Desktop/NVDALinearLogisticAnalysis/nvda_new.csv')
# data for nvda earnings
df_nvda_earnings = pd.read_csv('C:/Users/micha/OneDrive/Desktop/NVDALinearLogisticAnalysis/NVDA_earnings.csv')
# aux historical
df_aux = pd.read_csv('C:/Users/micha/OneDrive/Desktop/NVDALinearLogisticAnalysis/aux_historical_data.csv')
# spx historical
df_spx = pd.read_csv('C:/Users/micha/OneDrive/Desktop/NVDALinearLogisticAnalysis/spx_historical_data.csv')
# Display the first few rows of each dataframe to verify the data
print("Stock Data:")
print(df_stock.head())
print("\nTI Data:")
print(df_ti.head())


Stock Data:
   id symbol             datetime    open    high     low   close   volume
0   1   NVDA  2024-07-19 15:59:00  118.01  118.20  117.95  117.99  3212446
1   2   NVDA  2024-07-19 15:58:00  117.93  118.07  117.92  118.01  1192224
2   3   NVDA  2024-07-19 15:57:00  117.86  118.00  117.83  117.94   905114
3   4   NVDA  2024-07-19 15:56:00  117.87  117.90  117.80  117.86   770476
4   5   NVDA  2024-07-19 15:55:00  117.88  118.08  117.84  117.86  1034337

TI Data:
   id symbol           bydatetime     macd  bysignal  byhistogram     ema  \
0   1   NVDA  2024-07-19 15:59:00 -0.09599  -0.11612      0.02013  117.95   
1   2   NVDA  2024-07-19 15:58:00 -0.10684  -0.12115      0.01432  117.94   
2   3   NVDA  2024-07-19 15:57:00 -0.12065  -0.12473      0.00409  117.92   
3   4   NVDA  2024-07-19 15:56:00 -0.12939  -0.12575     -0.00364  117.92   
4   5   NVDA  2024-07-19 15:55:00 -0.13053  -0.12484     -0.00569  117.93   

     rsi  
0  46.19  
1  46.89  
2  43.18  
3  38.64  
4  38.82  

The files being read in above are results of the extract_from_sql_tool which taps into the MYSQL server and extracts the entirety of the table. These tables are not altered and so will contain all information gathered on that entity. Keeping the code above to run everytime is ok. 

*****TO DO***********
GENERATE CODE TO LIST NAMES OF ATTRIBUTES
UNIFY NAMES AND DATA TYPES
GENERATE CODE TO PULL MOST COMMON MINIMUM AND MAXIMUM DATES
TRIM AND MERGE


In [29]:

#print("\nColumns in NVDA Earnings Data:")
#print(df_nvda_earnings.columns)
#print("\nColumns in NVDA Info Data:")
#print(df_nvda_info.columns)
#print("\nColumns in NVDA Recent New Data:")
#print(df_recent_new.columns)

print("\nColumns in AUX historical:")
print(df_aux.columns)
print("\nColumns in SPX historical:")
print(df_spx.columns)
print("Columns in Stock Data:")
print(df_stock.columns)
print("\nColumns in Technical Indicators Data:")
print(df_ti.columns)




Columns in AUX historical:
Index(['datetime', 'open', 'high', 'low', 'close', 'macd', 'macd_signal',
       'macd_hist', 'ema', 'rsi'],
      dtype='object')

Columns in SPX historical:
Index(['datetime', 'open', 'high', 'low', 'close', 'volume', 'macd',
       'macd_signal', 'macd_hist', 'ema', 'rsi'],
      dtype='object')
Columns in Stock Data:
Index(['id', 'symbol', 'datetime', 'open', 'high', 'low', 'close', 'volume'], dtype='object')

Columns in Technical Indicators Data:
Index(['id', 'symbol', 'bydatetime', 'macd', 'bysignal', 'byhistogram', 'ema',
       'rsi'],
      dtype='object')


In [30]:
# Renaming columns for AUX
df_aux.rename(columns={
    "open": "aux_open", 
    "close": "aux_close", 
    "aux_high": 'high',
    "aux_low": 'low',
    "aux_close": 'close', 
    "aux_macd":'macd',
    "aux_macd_signal": 'macd_signal',
    "aux_macd_hist": 'macd_hist',
    "aux_ema": 'ema',
    "aux_rsi": 'rsi'
}, inplace=True)

# Renaming columns for SPX
df_spx.rename(columns={  
    "open": "spx_open", 
    "close": "spx_close", 
    "spx_high": 'high',
    "spx_low": 'low',
    "spx_close": 'close', 
    "spx_macd":'macd',
    "spx_macd_signal": 'macd_signal',
    "spx_macd_hist": 'macd_hist',
    "spx_ema": 'ema',
    "spx_rsi": 'rsi'
}, inplace=True)

df_stock.drop(columns=['id', 'symbol'], inplace=True)
df_ti.drop(columns=['id', 'symbol'], inplace=True)



In [31]:
df_stock_ti = pd.merge(df_stock, df_ti, on='datetime', how='inner')


KeyError: 'datetime'

In [38]:
# While experimenting downloading via api there are diplicates needing to be removed

duplicate_entries = df_stock['datetime'].duplicated().any()
if duplicate_entries:
    print("There are duplicate datetime entries.")
else:
    print("No duplicate datetime entries found.")

duplicate_entries = df_stock[df_stock['datetime'].duplicated(keep=False)]
print(duplicate_entries)


There are duplicate datetime entries.
                  datetime    open    high     low   close    volume
0      2024-07-19 15:59:00  118.01  118.20  117.95  117.99   3212446
1      2024-07-19 15:58:00  117.93  118.07  117.92  118.01   1192224
2      2024-07-19 15:57:00  117.86  118.00  117.83  117.94    905114
3      2024-07-19 15:56:00  117.87  117.90  117.80  117.86    770476
4      2024-07-19 15:55:00  117.88  118.08  117.84  117.86   1034337
...                    ...     ...     ...     ...     ...       ...
75060  2024-07-25 09:34:00  111.60  112.25  111.50  111.64   2567340
75061  2024-07-25 09:33:00  112.43  112.50  111.51  111.59   3665020
75062  2024-07-25 09:32:00  113.29  113.30  112.30  112.41   2248988
75063  2024-07-25 09:31:00  113.34  113.91  113.05  113.29   1471099
75064  2024-07-25 09:30:00  113.08  113.74  112.91  113.34  21406663

[10008 rows x 6 columns]


In [24]:
def print_df_columns(df_list):
    for df in df_list:
        print(f"Columns in: df")
        print(df.columns)
        print("\n")
dfs = [df_nvda_earnings, df_nvda_info, df_recent_new, df_stock, df_ti, df_aux, df_spx]
print_df_columns(dfs)

Columns in: df
Index(['date', 'time', 'eps_estimate', 'eps_actual', 'difference',
       'surprise_prc'],
      dtype='object')


Columns in: df
Index(['id', 'symbol', 'country', 'currency', 'exchange', 'ipo',
       'marketCapitalization', 'name', 'phone', 'shareOutstanding', 'ticker',
       'weburl', 'logo', 'finnhubIndustry'],
      dtype='object')


Columns in: df
Index(['id', 'symbol', 'datetime', 'price'], dtype='object')


Columns in: df
Index(['id', 'symbol', 'datetime', 'open', 'high', 'low', 'close', 'volume'], dtype='object')


Columns in: df
Index(['id', 'symbol', 'bydatetime', 'macd', 'bysignal', 'byhistogram', 'ema',
       'rsi'],
      dtype='object')


Columns in: df
Index(['datetime', 'open', 'high', 'low', 'close', 'macd', 'macd_signal',
       'macd_hist', 'ema', 'rsi'],
      dtype='object')


Columns in: df
Index(['datetime', 'open', 'high', 'low', 'close', 'volume', 'macd',
       'macd_signal', 'macd_hist', 'ema', 'rsi'],
      dtype='object')


